In [1]:
import numpy as np #importing numpy
import pandas as pd #importing pandas
import PIL #importing pillow for image processing
from PIL import Image#importing image from pillow library
import skimage#importing skimage library
import os#importing os library
import time#importing time library

#Capture start time of the program
start_time = time.time()

In [2]:
from string import digits
#code to remove numbers in a string and assigning it to the variable
r_digits = str.maketrans('', '', digits)
#taking source data folder path to a varible
rawdata_path = 'D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//128//data//'
#changing current librbay to source data
os.chdir(rawdata_path)
#taking sub-directories to a list
dirs = os.listdir()
directories = []
diseases = []
#Taking directories into a list and extract disease name from folder name
for x in dirs:
    res = x.translate(r_digits).lower()
    diseases.append(res)
    temp = rawdata_path + x + '//'
    directories.append(temp)

In [3]:
for i in directories:
    os.chdir(i)
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        image = Image.open(j,mode='r')
        size = (128,128)
        image = image.resize(size)
        image = image.save(j)

In [4]:
#creating different dataframes for saving data in different vector forms
#gray data
gray_df = pd.DataFrame(columns=['mean','disease'])
#rgb data
rgb_df = pd.DataFrame(columns=['mean','disease'])
#red data
red_df = pd.DataFrame(columns=['mean','disease'])
#green data
green_df = pd.DataFrame(columns=['mean','disease'])
#blue data
blue_df = pd.DataFrame(columns=['mean','disease'])

In [5]:
#extracting data from images
x = 0
#creating enpty directories to save the vector from in to a list
gray_test = []
rgb_final = []
red_final = []
green_final = []
blue_final = []
disease = []
#reading data directory by directory
for i in directories:
    os.chdir(i)
    k = diseases[x]
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        temp_list = []
        rgb_temp = []
        red_temp = []
        green_temp = []
        blue_temp = []
        gray = []
        data = []
        red = []
        green = []
        blue = []
        #opening the image in rgb format
        image = Image.open(j,mode='r')
        data = image.convert('RGB')
        data = np.array(data)
        #extracting only red components
        red = data[:,:,0]
        #extracting only green components
        green = data[:,:,1]
        #extracting only blue components
        blue = data[:,:,2]
        data = data.ravel()
        #saving rgb vector data into list
        rgb_final.append(data)
        rgb_temp.append(np.mean(data))
        rgb_temp.append(k)
        rgb_df.loc[len(rgb_df)] = rgb_temp
        #saving red channel data into dataframe and vector data to a list.
        red = red.ravel()
        red_final.append(red)
        red_temp.append(np.mean(red))
        red_temp.append(k)
        red_df.loc[len(red_df)] = red_temp
        #saving blue channel data into dataframe and vector data to a list.
        blue = blue.ravel()
        blue_final.append(blue)
        blue_temp.append(np.mean(blue))
        blue_temp.append(k)
        blue_df.loc[len(blue_df)] = blue_temp
        #saving green channel data into dataframe and vector data to a list.
        green = green.ravel()
        green_final.append(green)
        green_temp.append(np.mean(green))
        green_temp.append(k)
        green_df.loc[len(green_df)] = green_temp
        #opening the image in gray format
        gray = image.convert('L')
        gray = np.array(gray)
        gray = gray.ravel()
        gray_test.append(gray)
        temp_list.append(np.mean(gray))
        temp_list.append(k)       
        gray_df.loc[len(gray_df)] = temp_list
    x = x+1

In [6]:
#changing path
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//128//vector//')

In [7]:
#saving the lists as numpy arrays in the local system
np.save('gray_test',gray_test)
np.save('rgb_data',rgb_final)
np.save('red_data',red_final)
np.save('green_data',green_final)
np.save('blue_data',blue_final)

In [8]:
#saving the dataframe as csv files in the local system
gray_df.to_csv('gray_data.csv',index=False)
rgb_df.to_csv('rgb_data.csv',index=False)
red_df.to_csv('red_data.csv',index=False)
green_df.to_csv('green_data.csv',index=False)
blue_df.to_csv('blue_data.csv',index=False)

# KNN on rgb data

In [9]:
#reading numpy array data
rgb_test = np.load('rgb_data.npy')
#reading csv file
rgb_df = pd.read_csv('rgb_data.csv')

In [10]:
loaded = pd.DataFrame(rgb_test)

In [11]:
#assigning features and target
features = loaded.values
target = rgb_df['disease']
#splitting the data to 80 percent and 20 percent
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=24)
#scaling the data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [12]:
#K-Model on the data
from sklearn.neighbors import KNeighborsClassifier
kmodel = KNeighborsClassifier(n_neighbors=120)

In [13]:
#k-model fitting and accuracy
kmodel.fit(Ftrain,Ltrain)
print(kmodel.score(Ftest,Ltest))

0.5242518059855521


In [14]:
#importing classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
#printing results of the kmodel
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.31      0.80      0.45       345
                 early_blight       0.00      0.00      0.00       153
                      healthy       0.37      0.70      0.48       234
                  late_blight       0.46      0.36      0.40       309
                    leaf_mold       0.74      0.46      0.57       145
           septoria_leaf_spot       0.67      0.28      0.39       293
                  target_spot       0.61      0.14      0.23       240
          tomato_mosaic_virus       1.00      0.04      0.07        54
tomato_yellow_leaf_curl_virus       0.80      0.78      0.79       854
      two-spotted_spider_mite       0.53      0.45      0.49       280

                     accuracy                           0.52      2907
                    macro avg       0.55      0.40      0.39      2907
                 weighted avg       0.57      0.52      0.50      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on rgb data

In [16]:
#importing logistic data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
print(model.score(Ftest,Ltest))

0.7512899896800825


In [17]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.68      0.80      0.73       345
                 early_blight       0.42      0.37      0.39       153
                      healthy       0.82      0.72      0.76       234
                  late_blight       0.66      0.64      0.65       309
                    leaf_mold       0.69      0.68      0.68       145
           septoria_leaf_spot       0.75      0.73      0.74       293
                  target_spot       0.69      0.60      0.64       240
          tomato_mosaic_virus       0.72      0.78      0.75        54
tomato_yellow_leaf_curl_virus       0.91      0.92      0.92       854
      two-spotted_spider_mite       0.67      0.72      0.69       280

                     accuracy                           0.75      2907
                    macro avg       0.70      0.70      0.70      2907
                 weighted avg       0.75      0.75      0.75      2907

[[2

# SVM on RGB data

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7832817337461301


In [19]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.75      0.87      0.81       345
                 early_blight       0.44      0.38      0.41       153
                      healthy       0.84      0.81      0.83       234
                  late_blight       0.70      0.72      0.71       309
                    leaf_mold       0.70      0.70      0.70       145
           septoria_leaf_spot       0.75      0.73      0.74       293
                  target_spot       0.72      0.65      0.69       240
          tomato_mosaic_virus       0.81      0.87      0.84        54
tomato_yellow_leaf_curl_virus       0.92      0.93      0.92       854
      two-spotted_spider_mite       0.73      0.71      0.72       280

                     accuracy                           0.78      2907
                    macro avg       0.74      0.74      0.74      2907
                 weighted avg       0.78      0.78      0.78      2907

[[3

# Random Forest on RGB 

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,n_estimators=100, oob_score=True)

In [22]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [23]:
classifier_rf.oob_score_

0.5525636613902272

In [24]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.78      0.56      0.65       345
                 early_blight       0.00      0.00      0.00       153
                      healthy       0.63      0.80      0.71       234
                  late_blight       0.67      0.41      0.51       309
                    leaf_mold       0.75      0.43      0.54       145
           septoria_leaf_spot       0.58      0.48      0.53       293
                  target_spot       0.50      0.29      0.37       240
          tomato_mosaic_virus       0.00      0.00      0.00        54
tomato_yellow_leaf_curl_virus       0.50      0.98      0.66       854
      two-spotted_spider_mite       0.64      0.06      0.12       280

                     accuracy                           0.56      2907
                    macro avg       0.51      0.40      0.41      2907
                 weighted avg       0.56      0.56      0.50      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes on RGB data

In [25]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [26]:
y_pred = gnb.predict(Ftest)

In [27]:
from sklearn import metrics
metrics.accuracy_score(Ltest, y_pred)

0.49501203990368076

In [28]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.54      0.75      0.63       345
                 early_blight       0.24      0.30      0.27       153
                      healthy       0.39      0.67      0.49       234
                  late_blight       0.50      0.27      0.35       309
                    leaf_mold       0.53      0.46      0.49       145
           septoria_leaf_spot       0.45      0.44      0.45       293
                  target_spot       0.48      0.42      0.45       240
          tomato_mosaic_virus       0.18      0.76      0.29        54
tomato_yellow_leaf_curl_virus       0.89      0.48      0.63       854
      two-spotted_spider_mite       0.40      0.51      0.45       280

                     accuracy                           0.50      2907
                    macro avg       0.46      0.51      0.45      2907
                 weighted avg       0.58      0.50      0.51      2907

[[2

# Decision Tree on RGB data

In [29]:
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()

In [30]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4743722050223598

In [31]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.49      0.56      0.52       345
                 early_blight       0.12      0.14      0.13       153
                      healthy       0.55      0.56      0.56       234
                  late_blight       0.40      0.37      0.38       309
                    leaf_mold       0.33      0.34      0.33       145
           septoria_leaf_spot       0.44      0.42      0.43       293
                  target_spot       0.31      0.31      0.31       240
          tomato_mosaic_virus       0.12      0.17      0.14        54
tomato_yellow_leaf_curl_virus       0.71      0.66      0.68       854
      two-spotted_spider_mite       0.38      0.38      0.38       280

                     accuracy                           0.48      2907
                    macro avg       0.38      0.39      0.39      2907
                 weighted avg       0.48      0.48      0.48      2907

[[1

# KNN on grayscale data

In [32]:
#reading numpy array data
gray_test = np.load('gray_test.npy')
#reading csv file
gray_df = pd.read_csv('gray_data.csv')

In [33]:
loaded = pd.DataFrame(gray_test)

In [34]:
features = loaded.values
target = gray_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [35]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4368765049879601

In [36]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.77      0.47       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.35      0.59      0.44       258
                  late_blight       0.31      0.27      0.29       316
                    leaf_mold       0.71      0.38      0.50       145
           septoria_leaf_spot       0.58      0.30      0.40       262
                  target_spot       0.83      0.02      0.04       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.52      0.64      0.58       857
      two-spotted_spider_mite       0.59      0.30      0.40       251

                     accuracy                           0.44      2907
                    macro avg       0.42      0.33      0.31      2907
                 weighted avg       0.46      0.44      0.39      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on grayscale data

In [37]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.4021327829377365

In [38]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.49      0.43       349
                 early_blight       0.18      0.14      0.16       182
                      healthy       0.40      0.26      0.32       258
                  late_blight       0.30      0.32      0.31       316
                    leaf_mold       0.31      0.32      0.32       145
           septoria_leaf_spot       0.28      0.27      0.27       262
                  target_spot       0.25      0.17      0.20       229
          tomato_mosaic_virus       0.21      0.12      0.15        58
tomato_yellow_leaf_curl_virus       0.56      0.64      0.60       857
      two-spotted_spider_mite       0.36      0.37      0.36       251

                     accuracy                           0.40      2907
                    macro avg       0.32      0.31      0.31      2907
                 weighted avg       0.39      0.40      0.39      2907

[[1

# SVM on Grayscale data

In [39]:
clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4671482628138975


In [40]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.56      0.46       349
                 early_blight       0.24      0.25      0.24       182
                      healthy       0.52      0.49      0.51       258
                  late_blight       0.37      0.38      0.37       316
                    leaf_mold       0.42      0.44      0.43       145
           septoria_leaf_spot       0.38      0.35      0.37       262
                  target_spot       0.37      0.24      0.29       229
          tomato_mosaic_virus       0.29      0.17      0.22        58
tomato_yellow_leaf_curl_virus       0.64      0.63      0.63       857
      two-spotted_spider_mite       0.49      0.44      0.46       251

                     accuracy                           0.47      2907
                    macro avg       0.41      0.40      0.40      2907
                 weighted avg       0.47      0.47      0.46      2907

[[1

# Random Forest on Grayscale

In [41]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [42]:
classifier_rf.oob_score_

0.43392980041293877

In [43]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.63      0.48      0.54       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.69      0.33      0.44       258
                  late_blight       0.74      0.28      0.41       316
                    leaf_mold       1.00      0.03      0.05       145
           septoria_leaf_spot       0.59      0.45      0.51       262
                  target_spot       0.67      0.01      0.02       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.37      0.91      0.53       857
      two-spotted_spider_mite       0.57      0.17      0.26       251

                     accuracy                           0.44      2907
                    macro avg       0.53      0.27      0.28      2907
                 weighted avg       0.53      0.44      0.38      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on Grayscale

In [44]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [45]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3591331269349845

In [46]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.74      0.55       349
                 early_blight       0.23      0.28      0.25       182
                      healthy       0.32      0.62      0.42       258
                  late_blight       0.40      0.22      0.28       316
                    leaf_mold       0.45      0.32      0.38       145
           septoria_leaf_spot       0.37      0.48      0.42       262
                  target_spot       0.42      0.32      0.36       229
          tomato_mosaic_virus       0.11      0.57      0.18        58
tomato_yellow_leaf_curl_virus       0.62      0.13      0.22       857
      two-spotted_spider_mite       0.36      0.44      0.39       251

                     accuracy                           0.36      2907
                    macro avg       0.37      0.41      0.35      2907
                 weighted avg       0.44      0.36      0.34      2907

[[2

# Decision Tree on Grayscale

In [47]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.39697282421740626

In [48]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.44      0.45       349
                 early_blight       0.15      0.13      0.14       182
                      healthy       0.46      0.40      0.43       258
                  late_blight       0.38      0.33      0.35       316
                    leaf_mold       0.24      0.26      0.25       145
           septoria_leaf_spot       0.32      0.40      0.36       262
                  target_spot       0.25      0.24      0.24       229
          tomato_mosaic_virus       0.09      0.10      0.10        58
tomato_yellow_leaf_curl_virus       0.53      0.54      0.54       857
      two-spotted_spider_mite       0.41      0.41      0.41       251

                     accuracy                           0.40      2907
                    macro avg       0.33      0.33      0.33      2907
                 weighted avg       0.40      0.40      0.40      2907

[[1

# KNN on Red channel data

In [49]:
#reading numpy array data
red_test = np.load('red_data.npy')
#reading csv file
red_df = pd.read_csv('red_data.csv')

In [50]:
loaded = []
for i in red_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = red_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [51]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4688682490540076

In [52]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.72      0.52       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.34      0.69      0.45       258
                  late_blight       0.39      0.28      0.32       316
                    leaf_mold       0.67      0.38      0.48       145
           septoria_leaf_spot       0.61      0.28      0.39       262
                  target_spot       0.59      0.16      0.25       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.52      0.65      0.58       857
      two-spotted_spider_mite       0.58      0.49      0.53       251

                     accuracy                           0.47      2907
                    macro avg       0.41      0.36      0.35      2907
                 weighted avg       0.46      0.47      0.43      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on Red channel data

In [53]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.4437564499484004

In [54]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.56      0.50       349
                 early_blight       0.23      0.14      0.17       182
                      healthy       0.47      0.29      0.36       258
                  late_blight       0.32      0.35      0.34       316
                    leaf_mold       0.40      0.39      0.40       145
           septoria_leaf_spot       0.31      0.31      0.31       262
                  target_spot       0.29      0.20      0.24       229
          tomato_mosaic_virus       0.28      0.19      0.22        58
tomato_yellow_leaf_curl_virus       0.59      0.67      0.63       857
      two-spotted_spider_mite       0.43      0.44      0.43       251

                     accuracy                           0.44      2907
                    macro avg       0.37      0.36      0.36      2907
                 weighted avg       0.43      0.44      0.43      2907

[[1

# SVM on Red channel data

In [55]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4867561059511524


In [56]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.59      0.50       349
                 early_blight       0.25      0.21      0.23       182
                      healthy       0.56      0.47      0.51       258
                  late_blight       0.37      0.42      0.40       316
                    leaf_mold       0.47      0.45      0.46       145
           septoria_leaf_spot       0.38      0.39      0.38       262
                  target_spot       0.36      0.31      0.33       229
          tomato_mosaic_virus       0.30      0.21      0.24        58
tomato_yellow_leaf_curl_virus       0.65      0.63      0.64       857
      two-spotted_spider_mite       0.55      0.49      0.52       251

                     accuracy                           0.49      2907
                    macro avg       0.43      0.42      0.42      2907
                 weighted avg       0.49      0.49      0.49      2907

[[2

# Random forest on Red channel data

In [57]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4580178940123882

In [58]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.63      0.49      0.55       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.62      0.31      0.41       258
                  late_blight       0.70      0.31      0.43       316
                    leaf_mold       0.60      0.02      0.04       145
           septoria_leaf_spot       0.55      0.48      0.51       262
                  target_spot       0.86      0.08      0.14       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.40      0.89      0.55       857
      two-spotted_spider_mite       0.50      0.40      0.44       251

                     accuracy                           0.47      2907
                    macro avg       0.48      0.30      0.31      2907
                 weighted avg       0.51      0.47      0.41      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes on Red channel data

In [59]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [60]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3790849673202614

In [61]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.47      0.73      0.57       349
                 early_blight       0.22      0.24      0.23       182
                      healthy       0.32      0.68      0.43       258
                  late_blight       0.46      0.25      0.32       316
                    leaf_mold       0.40      0.33      0.36       145
           septoria_leaf_spot       0.36      0.44      0.39       262
                  target_spot       0.44      0.40      0.42       229
          tomato_mosaic_virus       0.13      0.62      0.22        58
tomato_yellow_leaf_curl_virus       0.64      0.14      0.23       857
      two-spotted_spider_mite       0.41      0.57      0.48       251

                     accuracy                           0.38      2907
                    macro avg       0.39      0.44      0.37      2907
                 weighted avg       0.46      0.38      0.36      2907

[[2

# Decision Tree on Red channel

In [62]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.41658066735466115

In [63]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.45      0.47      0.46       349
                 early_blight       0.12      0.10      0.11       182
                      healthy       0.48      0.45      0.46       258
                  late_blight       0.40      0.34      0.37       316
                    leaf_mold       0.29      0.32      0.31       145
           septoria_leaf_spot       0.30      0.37      0.34       262
                  target_spot       0.34      0.37      0.36       229
          tomato_mosaic_virus       0.08      0.09      0.08        58
tomato_yellow_leaf_curl_virus       0.55      0.54      0.55       857
      two-spotted_spider_mite       0.39      0.40      0.40       251

                     accuracy                           0.41      2907
                    macro avg       0.34      0.34      0.34      2907
                 weighted avg       0.42      0.41      0.41      2907

[[1

# KNN on green channel data

In [64]:
#reading numpy array data
green_test = np.load('green_data.npy')
#reading csv file
green_df = pd.read_csv('green_data.csv')
loaded = []
for i in green_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = green_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4238046095631235

In [65]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.28      0.78      0.42       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.34      0.54      0.42       258
                  late_blight       0.28      0.30      0.29       316
                    leaf_mold       0.62      0.39      0.47       145
           septoria_leaf_spot       0.60      0.34      0.43       262
                  target_spot       1.00      0.01      0.03       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.61      0.59      0.60       857
      two-spotted_spider_mite       0.54      0.29      0.38       251

                     accuracy                           0.42      2907
                    macro avg       0.43      0.32      0.30      2907
                 weighted avg       0.49      0.42      0.39      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on Green channel data

In [66]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.40144478844169246

In [67]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.47      0.42       349
                 early_blight       0.21      0.16      0.19       182
                      healthy       0.43      0.31      0.36       258
                  late_blight       0.30      0.31      0.30       316
                    leaf_mold       0.26      0.28      0.27       145
           septoria_leaf_spot       0.26      0.25      0.26       262
                  target_spot       0.20      0.13      0.16       229
          tomato_mosaic_virus       0.45      0.16      0.23        58
tomato_yellow_leaf_curl_virus       0.56      0.66      0.61       857
      two-spotted_spider_mite       0.34      0.34      0.34       251

                     accuracy                           0.40      2907
                    macro avg       0.34      0.31      0.31      2907
                 weighted avg       0.39      0.40      0.39      2907

[[1

# SVM on green channel data

In [68]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4458204334365325


In [69]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.37      0.57      0.45       349
                 early_blight       0.26      0.25      0.26       182
                      healthy       0.52      0.41      0.46       258
                  late_blight       0.36      0.36      0.36       316
                    leaf_mold       0.36      0.36      0.36       145
           septoria_leaf_spot       0.33      0.30      0.31       262
                  target_spot       0.31      0.22      0.26       229
          tomato_mosaic_virus       0.40      0.17      0.24        58
tomato_yellow_leaf_curl_virus       0.62      0.64      0.63       857
      two-spotted_spider_mite       0.43      0.38      0.41       251

                     accuracy                           0.45      2907
                    macro avg       0.40      0.37      0.37      2907
                 weighted avg       0.45      0.45      0.44      2907

[[1

# Random Forest on green channel data

In [70]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.42833792154163797

In [71]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.60      0.47      0.53       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.63      0.34      0.45       258
                  late_blight       0.70      0.27      0.39       316
                    leaf_mold       0.00      0.00      0.00       145
           septoria_leaf_spot       0.61      0.40      0.48       262
                  target_spot       0.00      0.00      0.00       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.36      0.92      0.52       857
      two-spotted_spider_mite       0.79      0.09      0.16       251

                     accuracy                           0.43      2907
                    macro avg       0.37      0.25      0.25      2907
                 weighted avg       0.43      0.43      0.36      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on Green channel data

In [72]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.34502923976608185

In [73]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.74      0.54       349
                 early_blight       0.24      0.33      0.28       182
                      healthy       0.34      0.58      0.43       258
                  late_blight       0.38      0.22      0.28       316
                    leaf_mold       0.38      0.28      0.33       145
           septoria_leaf_spot       0.31      0.50      0.39       262
                  target_spot       0.41      0.26      0.32       229
          tomato_mosaic_virus       0.11      0.55      0.18        58
tomato_yellow_leaf_curl_virus       0.54      0.11      0.19       857
      two-spotted_spider_mite       0.35      0.41      0.38       251

                     accuracy                           0.35      2907
                    macro avg       0.35      0.40      0.33      2907
                 weighted avg       0.41      0.35      0.32      2907

[[2

# Decision Tree on Green channel

In [74]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.40350877192982454

In [75]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.44      0.43       349
                 early_blight       0.16      0.14      0.15       182
                      healthy       0.51      0.50      0.50       258
                  late_blight       0.36      0.34      0.35       316
                    leaf_mold       0.19      0.21      0.20       145
           septoria_leaf_spot       0.37      0.41      0.39       262
                  target_spot       0.33      0.27      0.29       229
          tomato_mosaic_virus       0.11      0.12      0.11        58
tomato_yellow_leaf_curl_virus       0.57      0.56      0.56       857
      two-spotted_spider_mite       0.31      0.34      0.33       251

                     accuracy                           0.41      2907
                    macro avg       0.33      0.33      0.33      2907
                 weighted avg       0.41      0.41      0.41      2907

[[1

# KNN on blue channel data

In [76]:
#reading numpy array data
blue_test = np.load('blue_data.npy')
#reading csv file
blue_df = pd.read_csv('blue_data.csv')
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = blue_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4368765049879601

In [77]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.77      0.47       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.35      0.59      0.44       258
                  late_blight       0.31      0.27      0.29       316
                    leaf_mold       0.71      0.38      0.50       145
           septoria_leaf_spot       0.58      0.30      0.40       262
                  target_spot       0.83      0.02      0.04       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.52      0.64      0.58       857
      two-spotted_spider_mite       0.59      0.30      0.40       251

                     accuracy                           0.44      2907
                    macro avg       0.42      0.33      0.31      2907
                 weighted avg       0.46      0.44      0.39      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# logistic on blue channel data

In [78]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.4021327829377365

In [79]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.49      0.43       349
                 early_blight       0.18      0.14      0.16       182
                      healthy       0.40      0.26      0.32       258
                  late_blight       0.30      0.32      0.31       316
                    leaf_mold       0.31      0.32      0.32       145
           septoria_leaf_spot       0.28      0.27      0.27       262
                  target_spot       0.25      0.17      0.20       229
          tomato_mosaic_virus       0.21      0.12      0.15        58
tomato_yellow_leaf_curl_virus       0.56      0.64      0.60       857
      two-spotted_spider_mite       0.36      0.37      0.36       251

                     accuracy                           0.40      2907
                    macro avg       0.32      0.31      0.31      2907
                 weighted avg       0.39      0.40      0.39      2907

[[1

# SVM on blue channel data

In [80]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4671482628138975


In [81]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.56      0.46       349
                 early_blight       0.24      0.25      0.24       182
                      healthy       0.52      0.49      0.51       258
                  late_blight       0.37      0.38      0.37       316
                    leaf_mold       0.42      0.44      0.43       145
           septoria_leaf_spot       0.38      0.35      0.37       262
                  target_spot       0.37      0.24      0.29       229
          tomato_mosaic_virus       0.29      0.17      0.22        58
tomato_yellow_leaf_curl_virus       0.64      0.63      0.63       857
      two-spotted_spider_mite       0.49      0.44      0.46       251

                     accuracy                           0.47      2907
                    macro avg       0.41      0.40      0.40      2907
                 weighted avg       0.47      0.47      0.46      2907

[[1

# Random Forest on blue channel data

In [82]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.43392980041293877

In [83]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.63      0.48      0.54       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.69      0.33      0.44       258
                  late_blight       0.74      0.28      0.41       316
                    leaf_mold       1.00      0.03      0.05       145
           septoria_leaf_spot       0.59      0.45      0.51       262
                  target_spot       0.67      0.01      0.02       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.37      0.91      0.53       857
      two-spotted_spider_mite       0.57      0.17      0.26       251

                     accuracy                           0.44      2907
                    macro avg       0.53      0.27      0.28      2907
                 weighted avg       0.53      0.44      0.38      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive bayes on blue channel data

In [84]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3591331269349845

In [85]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.74      0.55       349
                 early_blight       0.23      0.28      0.25       182
                      healthy       0.32      0.62      0.42       258
                  late_blight       0.40      0.22      0.28       316
                    leaf_mold       0.45      0.32      0.38       145
           septoria_leaf_spot       0.37      0.48      0.42       262
                  target_spot       0.42      0.32      0.36       229
          tomato_mosaic_virus       0.11      0.57      0.18        58
tomato_yellow_leaf_curl_virus       0.62      0.13      0.22       857
      two-spotted_spider_mite       0.36      0.44      0.39       251

                     accuracy                           0.36      2907
                    macro avg       0.37      0.41      0.35      2907
                 weighted avg       0.44      0.36      0.34      2907

[[2

# Decision Tree on Blue channel

In [86]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4072927416580667

In [87]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.47      0.47       349
                 early_blight       0.17      0.12      0.14       182
                      healthy       0.49      0.45      0.47       258
                  late_blight       0.43      0.34      0.38       316
                    leaf_mold       0.22      0.26      0.24       145
           septoria_leaf_spot       0.32      0.38      0.34       262
                  target_spot       0.23      0.25      0.24       229
          tomato_mosaic_virus       0.03      0.03      0.03        58
tomato_yellow_leaf_curl_virus       0.54      0.55      0.54       857
      two-spotted_spider_mite       0.36      0.38      0.37       251

                     accuracy                           0.40      2907
                    macro avg       0.32      0.32      0.32      2907
                 weighted avg       0.40      0.40      0.40      2907

[[1

In [88]:
end_time = time.time()
print(end_time - start_time)

37418.40356707573
